# Scheduling Machine Learning Pipelines using Apache Airflow

In this workshop, you will use Airflow to schedule a basic machine learning pipeline. The workshop consists of 3 assignments.

1. Schedule a basic 'hello world' example on Airflow
2. Schedule a machine learning pipeline on Airflow
3. Improve the the pipeline by creating your own custom Airflow operator

Along the way, you will come across these blockquotes (as shown below). In there, we provide supplemental information related to the assignments you are doing. This information is not necessary to understand the assignment, but offers additional reading material for Aiflow enthousiasts.

> This is a blockquote with supplemental information!

You are provided with a random username for this workshop. This username is used to interact with the resources on AWS. Your user can be retrieved from the environment variable `WORKSHOP_USER`. Execute the cell below to find out your user

In [ ]:
import os

os.environ['WORKSHOP_USER']

## Assignment 1: Hello World

In this assignment, we are going to schedule a simple workflow on Airflow to get used with the concepts. Airflow uses the following concepts for worfklows:

- DAG ([directed acyclic graph](https://en.wikipedia.org/wiki/Directed_acyclic_graph)): a description of the order in which work should take place
- Operator: a class that acts as a template for carrying out some work
- Task: a parameterized instance of an operator
- Task Instance: a task that 
     1. has been assigned to a DAG and 
     2. has a state associated with a specific run of the DAG
     
In the image below, you can see an example of an Airflow DAG. Each box in the image represents a task. Arrows between the boxes indicate a dependency. For this DAG, task "run_after_loop" has to wait for "runme_0", "runme_1", and "runme_2", and "run_this_last" has to wait for "also_run_this" and "run_after_loop".
     
![](https://images.ctfassets.net/be04ylp8y0qc/5Q0bUH9SmNXdyiWpug1ArJ/040a27277e334849930ece4c60f2058a/1_tctnw7FxqwmcwWS1bdKHpw.png?fm=jpg)

The code below defines a DAG in Airflow. The DAG has the following properties:

- The DAG starts on 2019-11-27 and runs every day at midnight
- This DAG covers 2 tasks. 1 of them is an instance of [`PythonOperator`](https://github.com/apache/airflow/blob/1.10.4/airflow/operators/python_operator.py)s, and 1 is an instance of the [`BashOperator`](https://github.com/apache/airflow/blob/1.10.4/airflow/operators/bash_operator.py). Both tasks will simply print a word to the logs.
- Task 'print_hello' runs first. Then, task 'print_world' will run 

Inspect the code and try to understand how the properties above are defined:

In [1]:
import datetime
from airflow.models import DAG
from airflow.operators.python_operator import PythonOperator
from airflow.operators.bash_operator import BashOperator

with DAG(
    dag_id='hello_world',
    schedule_interval='@daily',
    start_date=datetime.datetime(2019, 11, 27) 
) as dag:
    print_hello_operator = PythonOperator(
        task_id='print_hello', 
        python_callable=print, 
        op_args=['hello'] 
    )
    
    print_world_operator = BashOperator(
        task_id='print_world',
        bash_command='echo world'
    )
    
    print_hello_operator >> print_world_operator

> In this DAG, we use de `'@daily'` schedule interval, which means that the DAG will run every day at midnight. For more complex scheduling, a [cron expression](https://en.wikipedia.org/wiki/Cron#CRON_expression) can be used. 

> The bitshift operator `>>` is overloaded for operators in Airflow. It is used to define dependencies between tasks in a DAG. For more information about defining relationships between tasks, check out the [bitshift composition](https://airflow.apache.org/concepts.html#bitshift-composition) and [relationship helper](https://airflow.apache.org/concepts.html#relationship-helper) in the Airflow documentation.

This is everything we need to do to define a DAG that can be scheduled on Airflow. Because DAGs are defined using Python, we have a lot of freedom in how we want to design our DAG. We could, for example, dynamically create tasks by looping over lists. Furthermore, defining your DAGs as code makes it it easy to keep track on their version in a source code management system. 

The next step is to actually run this example on Airflow. The Airflow scheduler periodically scans a folder, called 'the DagBag', for files that define DAGs. There is a folder in this jupyter notebook server, called `dags`, which is also present at the airflow scheduler via a network file system. Any python file containing a DAG definition that we put there, will be picked up by the scheduler.

- Copy the code snippet above. Go to the dags folder in the file explorer, press the 'New' button in the upper right corner, and create a **Text File**. Name it `hello_world.py`. The name of the file is flexible, but should end with `.py`. Paste the copied code inside.
- Go to the airflow UI, which is located at `/<your username>/airflow` on the address where you are currently on. You should see the DAG view, which looks similar to the screen below (with less DAGs shown). Refresh the page regularly, as this does not happen automatically. Your DAG should appear within a few seconds. 
![](https://airflow.apache.org/_images/dags.png)
- The DAG is turned off at first. Turn it on and frequently refresh the page to see what happens. In the 'Recent Tasks' and 'DAG Runs' columns, you will see the circles changing. Hover over them to see what they mean. After a while, only the leftmost, dark green circles are filled. They show that all tasks and DAGs have completed successfully. The start date of our DAG was 27-11-2019 (2 days ago), and we see 2 completed DAG runs. Airflow automatically processes historical DAG runs if a DAG's start date lies in the past. This is a very useful feature for processing historical data for example. 
- Click on 'hello_world' in the 'DAG' column. You are brought to the tree view, showing all DAG runs and task instances and their status. They are all dark green, meaning they all completed successfully. The tree view looks as follows.
![](https://airflow.apache.org/_images/tree.png)
- Press the lower left, dark green square, which is the 'print_hello' task instance of the DAG run of 27-11-2019. The task instance context menu with various actions that can be performed on the task instance will pop up. The popup is shown below. Press 'View Log'. This brings you to the logs produced by the task instance. We can also see that is has printed 'hello' somewhere in the logs, as we expected from this task instance.
![](https://airflow.apache.org/_images/context.png)
- Click on 'Graph View'. Here we can see the graphical representation of our DAG. We can see that 'print_hello' is followed by 'print_world', as we defined this in the DAG definition above. Below you can see an example graph view of a DAG.
![](https://airflow.apache.org/_images/graph.png)

Congratulations, you have scheduled your first DAG on Airflow, and learned the basic features of the Airflow UI. This DAG does not do anything useful yet though. In the next assignment, we will create a DAG that will process actual data.

## Assignment 2: Machine Learning Pipeline

In this assignment, we will create an Airflow DAG to schedule a basic machine learning pipeline. The pipeline will use the famous Iris dataset and consists of 2 steps:

1. Preprocess the dataset by adding some new features
2. Train a predictive model on the dataset

The goal is to schedule this training pipeline on a regular interval. This makes sure your model gets updated frequently with the latest data. 

> In this example, we will use the same dataset for every run, but in reality you would like to use a new dataset every time the pipeline is run. This can be done using Airflow's [templating](https://airflow.apache.org/macros.html) mechanism.

You are provided with 2 scripts located in the folder `transform_scripts`. Each script transforms an input file and stores the transformed data in an output file. The locations of the input and output files are provided as arguments to the scripts. The first script, `preprocess.py`, takes a CSV with raw training data, and outputs a CSV with preprocessed training data. The second script, `train.py`, takes a CSV with preprocessed training data, and outputs a pickled machine learning model. Also, you are provided with an [S3 bucket](https://aws.amazon.com/s3/) containing our raw training data. S3 is a simple storage service provided available on Amazon AWS, allowing one to store and retrieve files in a scalable way. Let's use the [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html?id=docs_gateway) python package to list the resources contained in our S3 bucket:

In [ ]:
import boto3
import os

def list_s3():
    user = os.environ['WORKSHOP_USER']
    bucket = f'pydata-eindhoven-2019-airflow-{user}'

    client = boto3.client('s3', region_name='eu-west-1')

    return [o['Key'] for o in client.list_objects(Bucket=bucket)['Contents']]

list_s3()

We can see that the raw training data is already present in our bucket, along with some raw unlabeled data. Since this DAG should train a machine learning model, we are going to ignore the unlabeled data for now. Our DAG should to the following:

1. Retrieve the raw training data from S3, apply the `preprocess.py` transform script to it, and send the preprocessed CSV back to S3
2. Retrieve the preprocessed training data from S3, apply the `train.py` transform script to it, and send the pickled model to S3

Airflow's built-in [`S3FileTransformOperator`](https://github.com/apache/airflow/blob/1.10.4/airflow/operators/s3_file_transform_operator.py) will do exactly what we want. The operator does the following things:

1. Download a file from S3 to a temporary file on the Airflow worker machine
2. Apply a transform script to the file, producing a new, temporary file
3. Upload the output file to S3

Inspect the DAG below. It uses the `S3FileTransformOperator` to first preprocess the data, and then train the model. We see that the connection ID's to S3 are set to `'s3'`. The Airflow [connections](https://airflow.apache.org/howto/connection/index.html) feature allows one to securely store connection information to external systems in the Airflow database. The connection to your bucket is already setup for you, and has the name 's3'. This name is specified in the operators, to let Airflow know which connection to use. 

Your own S3 bucket for this workshop is already configured in the DAG definition. As in assignment 1, copy the contents of the cell below. In the `dags` folder, create a new file called `ml_pipeline.py`, and paste the copied content inside. This DAG also uses the transform scripts. Therefore, we also need to copy the `transform_scripts` folder into the `dags` folder. This can be done by executing the cell below.

> Usually, you want to avoid adding a lot of files that do not define DAGs to your DAG folder. Airflow scans this folder regularly to load new DAGs. The more files are in there, the bigger the performance hit will be. Consider storing files like transform scripts outside of the DAGs folder, or use the [`.airflowignore`](https://airflow.apache.org/concepts.html#airflowignore) file, which works similarly to a `.gitignore` file. You could even install your python dependencies on the Airflow host using pip.

In [ ]:
!cp -R transform_scripts dags/transform_scripts

In [ ]:
import datetime
from airflow.models import DAG
from airflow.operators.s3_file_transform_operator import S3FileTransformOperator
import os

user = os.environ['WORKSHOP_USER']
bucket = f'pydata-eindhoven-2019-airflow-{user}'
dag_folder = os.path.dirname(os.path.abspath(__file__))

with DAG(
    dag_id='ml_pipeline',
    schedule_interval='@daily',
    start_date=datetime.datetime(2019, 11, 27)
) as dag:
    preprocess_operator = S3FileTransformOperator(
        task_id='preprocess',
        transform_script=f'{dag_folder}/transform_scripts/preprocess.py',
        source_s3_key=f's3://{bucket}/raw_training_data.csv',
        dest_s3_key=f's3://{bucket}/preprocessed_training_data.csv',
        source_aws_conn_id='s3',
        dest_aws_conn_id='s3',
        replace=True
    )

    train_operator = S3FileTransformOperator(
        task_id='train',
        transform_script=f'{dag_folder}/transform_scripts/train.py',
        source_s3_key=f's3://{bucket}/preprocessed_training_data.csv',
        dest_s3_key=f's3://{bucket}/trained_model.pkl',
        source_aws_conn_id='s3',
        dest_aws_conn_id='s3',
        replace=True
    )

    preprocess_operator >> train_operator

In the Airflow UI, on port 8080, you should see your second DAG appear very soon. As in the first assignment, turn it on and refresh regularly to see the DAG runs succeed.

> We ran this pipeline on a single machine for this workshop. For large datasets or a large number of jobs, this does not scale well. Since Airflow is not a big data processing tool, a recommendation is to push computation to external systems as much as possible. For this example, we could decide to run our workload for example [AWS Sagemaker](https://aws.amazon.com/sagemaker/). 
>
> Also, Airflow provides various options for making it scalable. An example is the Kubernetes executor, which will spawn Kubernetes pods for every task instances ran. This allows Airflow to schedule a large number of parallel tasks, with as much resources as required for the task. To get started with this, check out [this blog](https://towardsdatascience.com/kubernetesexecutor-for-airflow-e2155e0f909c) by Brecht de Vlieger.

Now that the DAG runs are finished, we can check which files are generated on S3. We can use the previously created `list_s3` function to do so:

In [ ]:
list_s3()

As we can see, the raw data files are there, as well as the newly created preprocessed data and pickled model files. Since we kept the file names constant, the second DAG run has overwritten the files of the first DAG run. 

In this exercise, we have scheduled a machine learning pipeline, which transforms raw training data into a trained model. This model could now be served somewhere (i.e. using a web service), or used to make batch predictions (i.e. with Airflow).

The next assignment is a bonus assignment, where you will improve the pipeline above by creating your own Airflow operator.

## \[BONUS\] Assignment 3: Custom Operator

In the previous assigment, we have used the `S3FileTransformOperator` to transform our raw training data into a trained model. A logical next step would be that we use the trained model to make some batch predictions. In order to achieve this, we need to add 2 tasks to our previous DAG:

1. Preprocess the raw unlabeled data (to add the new features to it)
2. Use the trained model to make predictions on the preprocessed unlabeled data

The first task looks a lot like the preprocessing step of our training data, and should be trivial to add. For the second task, we need to download 2 files from S3 (the unlabeled data and the model), and need to use the model to make a prediction (using the scikit-learn estimator's `.predict` method). Both of these steps are new, and cannot easily be achieved by using the `S3FileTransformOperator` again. Luckily, Airflow provides an easy way to create custom operator.

Create your own operator for the prediction class. Let's take the `S3FileTransformOperator` code as inspiration. The code can be found [here](https://github.com/apache/airflow/blob/1.10.4/airflow/operators/s3_file_transform_operator.py). An Airflow operator has a few properties:

- It inherits from `BaseOperator`.
- It has an `__init__` method, where all inputs are defined.
- It overrides the `execute` method of `BaseOperator`, which is the method that gets executed when the task instance is triggered.
- It often uses one or more hooks to communicate with external services. In our case, this is the [`S3Hook`](https://github.com/apache/airflow/blob/1.10.4/airflow/hooks/S3_hook.py).

Once you created the operator, extend the DAG from assignment 2 with the 2 tasks specified above. Like before, copy the DAG into a text file in the `dags` folder, and inspect the result in the Airflow UI. A backbone for your DAG is provided below. After the DAG has finished running, check the results using the `list_s3` function. In case you get stuck, an example answer is stored in the `answers` folder.

In [ ]:
import datetime
from airflow.models import DAG, BaseOperator
from airflow.operators.s3_file_transform_operator import S3FileTransformOperator
from airflow.exceptions import AirflowException
from airflow.hooks.S3_hook import S3Hook
from airflow.utils.decorators import apply_defaults
import sys
import os
from tempfile import NamedTemporaryFile
import pandas as pd
import pickle


class S3PredictionOperator(BaseOperator):
    template_fields = ('source_s3_key', 'dest_s3_key')
    template_ext = ()
    ui_color = '#f9c915'

    @apply_defaults
    def __init__(
            self,
            source_s3_key,
            model_s3_key,
            dest_s3_key,
            aws_conn_id='aws_default',
            verify=None,
            replace=False,
            *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.source_s3_key = source_s3_key
        self.model_s3_key = model_s3_key
        self.dest_s3_key = dest_s3_key
        self.aws_conn_id = aws_conn_id
        self.verify = verify
        self.replace = replace
        self.output_encoding = sys.getdefaultencoding()

    def execute(self, context):
        # TODO: create your execute function. You can ignore the context parameter

        
user = os.environ['WORKSHOP_USER']
bucket = f'pydata-eindhoven-2019-airflow-{user}'
dag_folder = os.path.dirname(os.path.abspath(__file__))

with DAG(
    dag_id='custom_operator',
    schedule_interval='@daily',
    start_date=datetime.datetime(2019, 11, 27)
) as dag:
    preprocess_train_operator = S3FileTransformOperator(
        task_id='preprocess_train',
        transform_script=f'{dag_folder}/transform_scripts/preprocess.py',
        source_s3_key=f's3://{bucket}/raw_training_data.csv',
        dest_s3_key=f's3://{bucket}/preprocessed_training_data.csv',
        source_aws_conn_id='s3',
        dest_aws_conn_id='s3',
        replace=True
    )

    train_operator = S3FileTransformOperator(
        task_id='train',
        transform_script=f'{dag_folder}/transform_scripts/train.py',
        source_s3_key=f's3://{bucket}/preprocessed_training_data.csv',
        dest_s3_key=f's3://{bucket}/trained_model.pkl',
        source_aws_conn_id='s3',
        dest_aws_conn_id='s3',
        replace=True
    )
    
    # TODO preprocess your unlabeled data
    
    # TODO make predictions on your unlabeled data
    
    preprocess_train_operator >> train_operator
    # TODO create the correct dependencies for your newly created operators

In this assignment you created your own Airflow operator. The only thing you needed to do was to inherit from `BaseOperator` and override the `execute` method. By using this approach, we created an operator that makes predictions for us on unlabeled data. Additionally, we extended the DAG from assignment 2 with more tasks, resulting in a DAG with more complex dependencies.

## Summary

In this workshop, you learned what Airflow is, how to schedule your own Workflows with it, and how to use it for training machine learning models. As a bonus, you learned how to create your own operators that hold custom logic for the tasks you need to execute. This was just the tip of the iceberg, as the Airflow ecosystem offers lots of other features for creating production-grade data processing pipelines. This notebook contains various resources for further reading, in case you are interested. For questions, feel free to reach out to us.

Axel Goblet (axel [dot] goblet [at] bigdatarepublic [dot] nl)

Dick Abma (dick [dot] abma [at] bigdatarepublic [dot] nl)

https://www.bigdatarepublic.nl/

![](https://www.bigdatarepublic.nl/wp-content/uploads/2019/05/BDR_Logo_RGB_no_whitespace.jpg)